In [1]:
import processors
print(processors.__version__)

3.0.3


In [ ]:
from json import *

In [2]:
from processors import *
from processors.visualization import JupyterVisualizer as viz

API = ProcessorsAPI(port=8881, keep_alive=True, jar_path = "/Users/manningh/miniconda3/lib/python3.5/site-packages/processors/processors-server.jar")

INFO - Connection with server established!
INFO - Server version meets recommendations (v3.0.2)


In [6]:
import os

# abstracts_dir = "/Users/gus/repos/ohsu-hackathon/OHSU_NLP_Hackathon/sentences_and_abstracts/CIViC_abstracts"
abstracts_dir = "/Users/manningh/PycharmProjects/OHSU_NLP_Hackathon/sentences_and_abstracts/CIViC_abstracts"
serialized_documents_dir = "/Users/manningh/PycharmProjects/OHSU_NLP_Hackathon/sensitivity-and-adverse-outcomes/serialized-abstracts"

def annotate_documents(collection_dir):
    for f in os.listdir(collection_dir):
        file_path = os.path.join(collection_dir, f)
        text = open(file_path).read()
        doc = API.bionlp.annotate(text) 
        # use filename as the doc's id
        doc.id = os.path.splitext(f)[0]
        yield doc

In [5]:
#docs = annotate_documents(abstracts_dir)

In [9]:
def serialize_documents(collection):
    for doc in collection:
        #doc.id = os.path.splitext(doc.id)[0]
        with open("{}.json".format(doc.id), "w") as out_file:
            out_file.write(doc.to_JSON())

In [10]:
#serialize_documents(docs)

In [11]:
import json

def deserialize_doc_collection(serialized_docs_dir):
    for json_file in os.listdir(serialized_docs_dir):
        fpath = os.path.join(serialized_docs_dir, json_file)
        with open(fpath) as infile:
            text = infile.read()
        doc_dict = json.loads(text)
        doc = Document.load_from_JSON(doc_dict)
        # FIXED, but not released (ADD TEST)
        # py-processors v3.0.3 wasn't retrieving the doc id
        doc.id = os.path.splitext(json_file)[0]
        yield doc

In [12]:
abstracts = list(deserialize_doc_collection(serialized_documents_dir))

In [13]:
rules = open("rules.yml", "r")

def extract_from_collection(collection):
    for doc in collection:
        mentions = API.odin.extract_from_document(doc, rules)
        for m in sorted(mentions, key=lambda x: (x.document.id, x.sentence, x.start), reverse=False):
            yield m

In [14]:
def display_mentions(mentions):
    for m in mentions:
        print(m.document.id)
        viz.display_mention(m)

In [15]:
mentions = extract_from_collection(abstracts)

In [ ]:
#API.stop_server()